In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import os
import json
from pandas.io.json import json_normalize
import pprint
# import ijson
import matplotlib.pyplot as plt
import string
import fnmatch
import glob

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
for dirname, _, filenames in os.walk('C:/Users/trivikram.cheedella/OneDrive - JD Power/Data Science Data/CORD-19-research-challenge'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        pass

# for filename in fnmatch.filter(filenames, '*.txt'):
# Any results you write to the current directory are saved as output.

In [2]:
input = Path('C:/Users/trivikram.cheedella/OneDrive - JD Power/Data Science Data/CORD-19-research-challenge')
output = Path('C:/Users/trivikram.cheedella/OneDrive - JD Power/Data Science Data/CORD-19-research-challenge/output')

In [3]:
df_all_sources_metadata = pd.read_csv(input / 'metadata.csv')

In [4]:
print(df_all_sources_metadata.shape)
df_all_sources_metadata.info()

(44220, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44220 entries, 0 to 44219
Data columns (total 15 columns):
sha                            28462 non-null object
source_x                       44220 non-null object
title                          43996 non-null object
doi                            40750 non-null object
pmcid                          23319 non-null object
pubmed_id                      22943 non-null float64
license                        44220 non-null object
abstract                       35806 non-null object
publish_time                   34197 non-null object
authors                        41074 non-null object
journal                        33173 non-null object
Microsoft Academic Paper ID    964 non-null float64
WHO #Covidence                 1767 non-null object
has_full_text                  44220 non-null bool
full_text_file                 32829 non-null object
dtypes: bool(1), float64(2), object(12)
memory usage: 4.8+ MB


In [5]:
df_all_sources_metadata.head()

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file
0,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535.0,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,custom_license
1,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850.0,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",American Heart Journal,NaN,NaN,False,custom_license
2,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701.0,els-covid,NaN,1980-03-31,"Friedman, Gary D",American Heart Journal,NaN,NaN,False,custom_license
3,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077.0,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",The American Journal of Medicine,NaN,NaN,True,custom_license
4,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285.0,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",The American Journal of Medicine,NaN,NaN,False,custom_license


The following code for reading in the JSON file is taken from ...

https://www.kaggle.com/ivanegapratama/covid-eda-initial-exploration-tool

In [6]:
%%time
all_json = glob.glob(f'{input}/**/*.json', recursive=True)
len(all_json)

Wall time: 708 ms


29315

In [7]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
            # Extend Here
            #
            #
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'
first_row = FileReader(all_json[0])
print(first_row)

0015023cc06b5362d332b3baf348d11567ca2fbb: word count: 194 22 Text word count: 5168 23 24 25 author/funder. All rights reserved. No reuse allowed without permission. Abstract 27 The positive stranded RNA genomes of picornaviruses comprise a si... VP3, and VP0 (which is further processed to VP2 and VP4 during virus assembly) (6). The P2 64 and P3 regions encode the non-structural proteins 2B and 2C and 3A, 3B (1-3) (VPg), 3C pro and 4 structura...


In [8]:
%%time
dict_ = {'paper_id': [], 'abstract': [], 'body_text': []}
for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text'])
df_covid.head()

Processing index: 0 of 29315
Processing index: 2931 of 29315
Processing index: 5862 of 29315
Processing index: 8793 of 29315
Processing index: 11724 of 29315
Processing index: 14655 of 29315
Processing index: 17586 of 29315
Processing index: 20517 of 29315
Processing index: 23448 of 29315
Processing index: 26379 of 29315
Processing index: 29310 of 29315
Wall time: 2min 44s


,paper_id,abstract,body_text
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 Text word count: 5168 23 24...,"VP3, and VP0 (which is further processed to VP..."
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,,The 2019-nCoV epidemic has spread across China...
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,Infectious bronchitis (IB) causes significant ...,"Infectious bronchitis (IB), which is caused by..."
3,0139ea4ca580af99b602c6435368e7fdbefacb03,Nipah Virus (NiV) came into limelight recently...,Nipah is an infectious negative-sense single-s...
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,Background: A novel coronavirus (2019-nCoV) em...,"In December 2019, a cluster of patients with p..."


In [9]:
dict_ = None

In [10]:
%%time
df_covid['abstract_word_count'] = df_covid['abstract'].apply(lambda x: len(x.strip().split()))
df_covid['body_word_count'] = df_covid['body_text'].apply(lambda x: len(x.strip().split()))
df_covid.head()

Wall time: 21.4 s


,paper_id,abstract,body_text,abstract_word_count,body_word_count
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 Text word count: 5168 23 24...,"VP3, and VP0 (which is further processed to VP...",241,1728
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,,The 2019-nCoV epidemic has spread across China...,0,755
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,Infectious bronchitis (IB) causes significant ...,"Infectious bronchitis (IB), which is caused by...",1647,4003
3,0139ea4ca580af99b602c6435368e7fdbefacb03,Nipah Virus (NiV) came into limelight recently...,Nipah is an infectious negative-sense single-s...,326,2399
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,Background: A novel coronavirus (2019-nCoV) em...,"In December 2019, a cluster of patients with p...",22,4642


In [11]:
%%time
df_covid['abstract'] = df_covid['abstract'].apply(lambda x: x.lower())
df_covid['body_text'] = df_covid['body_text'].apply(lambda x: x.lower())
df_covid.head()

Wall time: 15 s


,paper_id,abstract,body_text,abstract_word_count,body_word_count
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 text word count: 5168 23 24...,"vp3, and vp0 (which is further processed to vp...",241,1728
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,,the 2019-ncov epidemic has spread across china...,0,755
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,infectious bronchitis (ib) causes significant ...,"infectious bronchitis (ib), which is caused by...",1647,4003
3,0139ea4ca580af99b602c6435368e7fdbefacb03,nipah virus (niv) came into limelight recently...,nipah is an infectious negative-sense single-s...,326,2399
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,background: a novel coronavirus (2019-ncov) em...,"in december 2019, a cluster of patients with p...",22,4642


In [12]:
df_covid.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
paper_id,29315,29315,b3935c8b90d9ca6671636eaae86eab9cb0295fe8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abstract,29315,21049,,8051,NaN,NaN,NaN,NaN,NaN,NaN,NaN
body_text,29315,29117,"in previous reports, workers have characterize...",4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abstract_word_count,29315,NaN,NaN,NaN,164.501,175.353,0,0,161,240,4767
body_word_count,29315,NaN,NaN,NaN,4802.29,7322.71,1,2444,3730,5509.5,260378


In [13]:
df_covid.drop_duplicates(['body_text'], inplace=True)
df_covid.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
paper_id,29117,29117,b3935c8b90d9ca6671636eaae86eab9cb0295fe8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abstract,29117,21044,,8004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
body_text,29117,29117,it has been recognized for decades that the co...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abstract_word_count,29117,NaN,NaN,NaN,164.504,175.637,0,0,160,240,4767
body_word_count,29117,NaN,NaN,NaN,4811.94,7344.37,1,2450,3733,5513,260378


In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS

In [15]:
lemmatizer = WordNetLemmatizer()

print("Number of stopwrods from STOPWORDS: ", len(STOPWORDS))
print("Number of stopwrods from stopwords.words('english'): ", len(stopwords.words('english')))
other_stopwords = ['q', 'license', 'preprint', 'copyright', 'http', 'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'fig', 'fig.', 'al.', 'di', 'la', 'il', 'del', 'le', 'della', 'dei', 'delle', 'una', 'da',  'dell',  'non', 'si']
custom_stopwords = []
custom_stopwords = list(set(stopwords.words('english') + list(STOPWORDS))) + other_stopwords

print("Number of stopwrods from custom_stopwords: ", len(custom_stopwords))
print(custom_stopwords)

Number of stopwrods from STOPWORDS:  190
Number of stopwrods from stopwords.words('english'):  179
Number of stopwrods from custom_stopwords:  263
['else', 're', 'http', 'and', 'will', 'needn', 'at', "here's", "isn't", "let's", 'theirs', "it's", "that's", 'under', 'com', 'also', 'below', 'such', 'after', "i've", 'as', 'being', 'be', 'so', 'doesn', 'these', "needn't", "he'd", 'because', 'now', 'ought', 'nor', 'than', 'our', 'haven', 'wasn', "he's", 'aren', "haven't", 'ourselves', "won't", "when's", "they'll", 'since', 'it', 'about', "we're", 'ever', 'been', 'which', 'until', 'before', 'very', 'with', 'when', 'from', 'hers', 'again', "she'd", 'while', 'above', 'both', "should've", 'what', 'shan', 't', 'up', 'here', 'did', 'during', 'how', 'her', 'had', 'or', 'same', 'its', "doesn't", 'no', 'y', "hasn't", "she'll", 'over', "weren't", 'you', 'should', 'this', 'if', 'just', "aren't", 'do', 'hadn', "mustn't", 'some', 'yours', 'but', 'by', 'don', 'between', 'myself', 'further', "we'll", 'ours

In [16]:
def remove_punc_stopword_tokens(tokens, tolower=False):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    remove_punc = [word for word in tokens if word not in string.punctuation]
    if tolower:
        return [word.lower() for word in remove_punc if word.lower() not in custom_stopwords]
    else:
        return [word for word in remove_punc if word not in custom_stopwords]

In [17]:
def remove_punc_stopword_text(text):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    remove_punc = [word for word in text if word not in string.punctuation]
    remove_punc = ''.join(remove_punc)
    return [word.lower() for word in remove_punc.split() if word.lower() not in custom_stopwords]

In [18]:
def clean_the_text(text):
        tokens = word_tokenize(text)
        remove_punc = [word for word in tokens if word not in string.punctuation]
        remove_stopwords = [word.lower() for word in remove_punc if word.lower() not in custom_stopwords]
        more_than_three = [w for w in remove_stopwords if len(w)>3]
        lem = [lemmatizer.lemmatize(w) for w in more_than_three]
        return ' '.join(lem)

# cleaned_text = clean_the_text("my name is TRIVIKRAM, and I am first born in my fam don't process processing patients")
# print(cleaned_text)

In [19]:
df_covid_for_nlp = df_covid.copy()
df_covid = None

In [20]:
%%time
df_covid_for_nlp['cleaned_text'] = df_covid_for_nlp['body_text'].apply(lambda x: clean_the_text(x))
# print(cleaned_text[:20])

Wall time: 1h 6min 10s


In [21]:
df_covid_for_nlp['cleaned_text'].head()

0    processed virus assembly region encode non-str...
1    2019-ncov epidemic spread across china country...
2    infectious bronchitis caused infectious bronch...
3    nipah infectious negative-sense single-strande...
4    december 2019 cluster patient pneumonia unknow...
Name: cleaned_text, dtype: object

In [121]:
df_covid_for_nlp_10K = df_covid_for_nlp.head(10000).copy()
df_covid_for_nlp_10K.shape

(10000, 6)

# Topic Modeling using Latent Dirichlet Allocation (LDA)

In [23]:
%%time

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix_countvectorizer = count_vect.fit_transform(df_covid_for_nlp_10K['cleaned_text'].values.astype('U'))

Wall time: 6min 14s


In [24]:
doc_term_matrix_countvectorizer

<10000x110226 sparse matrix of type '<class 'numpy.int64'>'
	with 7517951 stored elements in Compressed Sparse Row format>

In [25]:
%%time

from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=50, random_state=42, n_jobs=-1)
LDA.fit(doc_term_matrix_countvectorizer)

Wall time: 9min 3s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=50, n_jobs=-1,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [26]:
import random

for i in range(10):
    random_id = random.randint(0,len(count_vect.get_feature_names()))
    print(count_vect.get_feature_names()[random_id])

poult
vaccinepreventable
ctni
heading
strewn
denner
okayama
3050
ipilimumab
2673


In [27]:
first_topic = LDA.components_[0]

In [28]:
top_topic_words = first_topic.argsort()[-10:]
top_topic_words

array([106325,  96144,  76161,  86412, 105990,  53255, 106475,  23334,
        63943,  82451], dtype=int64)

In [29]:
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

viral
structure
pcv2
replication
vesicle
infected
virus
cell
membrane
prrsv


In [30]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['viral', 'structure', 'pcv2', 'replication', 'vesicle', 'infected', 'virus', 'cell', 'membrane', 'prrsv']


Top 10 words for topic #1:
['stem', 'ribosome', 'frame', 'trna', 'frameshifting', 'sequence', 'translation', 'site', 'mrna', 'codon']


Top 10 words for topic #2:
['cell', 'genome', 'interaction', 'ifitm3', 'host', 'replication', 'domain', 'viral', 'virus', 'protein']


Top 10 words for topic #3:
['method', 'positive', 'probe', 'reaction', 'pcr', 'rt', 'detection', 'primer', 'assay', 'sample']


Top 10 words for topic #4:
['host', 'immune', 'activation', 'ifn', 'type', 'protein', 'response', 'viral', 'virus', 'cell']


Top 10 words for topic #5:
['protein', 'vector', 'human', 'immune', 'virus', 'antigen', 'antibody', 'response', 'cell', 'vaccine']


Top 10 words for topic #6:
['anti', 'immune', 'level', 'macrophage', 'response', 'cytokine', 'expression', 'mouse', 'il', 'cell']


Top 10 words for topic #7:
['coronaviruses', 'hcov', 'coronavirus', 'camel

In [32]:
topic_values_countvectorizer = LDA.transform(doc_term_matrix_countvectorizer)
topic_values_countvectorizer.shape

(10000, 50)

In [33]:
%%time 

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix_Tfidf = tfidf_vect.fit_transform(df_covid_for_nlp_10K['cleaned_text'].values.astype('U'))

Wall time: 5min 35s


In [35]:
doc_term_matrix_Tfidf

<10000x110226 sparse matrix of type '<class 'numpy.float64'>'
	with 7517951 stored elements in Compressed Sparse Row format>

In [36]:
LDA_Tfidf = LatentDirichletAllocation(n_components=50, random_state=42, n_jobs=-1)
LDA_Tfidf.fit(doc_term_matrix_Tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=50, n_jobs=-1,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [37]:
for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(count_vect.get_feature_names()[random_id])

discontinuity
gucov
masstag
castillo
naturalproduct
1034
sublineage
barratt
hlas
cynomops


In [72]:
first_topic = LDA_Tfidf.components_[0]

In [39]:
top_topic_words = first_topic.argsort()[-10:]
top_topic_words

array([ 76746,  44963,  46644,  24298, 100642,  74849,  59485,  85971,
        43563,  82657], dtype=int64)

In [73]:
for i in top_topic_words:
#     print(first_topic[i])
    print(tfidf_vect.get_feature_names()[i])

per1
gnp
hace1
chikf
tocs
pakit03
letx
reinitiation
gemcitabine
psgs


In [43]:
for i,topic in enumerate(LDA_Tfidf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['per1', 'gnp', 'hace1', 'chikf', 'tocs', 'pakit03', 'letx', 'reinitiation', 'gemcitabine', 'psgs']


Top 10 words for topic #1:
['dcpep', 'coe', 'odns', 'gadd45b', 'l393', 'drill', 'ppg', 'egcg', 'egr', 'emodin']


Top 10 words for topic #2:
['whcv', 'sod1', 'cpps', 'dnsp16', 'restv', 'igf', 'iecs', 'cccdna', 'sudv', 'spherule']


Top 10 words for topic #3:
['ifi204', 'outsourcing', 'mdi', 'anguilla', 'rhmpv', 'rbp', 'serinc5', 'adam17', 'gx', 'pact']


Top 10 words for topic #4:
['vhhs', 'avan', 'samhd1', 'ridd', 'hsp27', 'ccovs', 'eplex', 'binase', 'ire1α', 'grp78']


Top 10 words for topic #5:
['il', 'cov', 'response', 'expression', 'antibody', 'viral', 'mouse', 'protein', 'virus', 'cell']


Top 10 words for topic #6:
['eif2α', 'hepg2', 'demyelination', 'cathepsin', 'astrovirus', 'orf2', 'astrocyte', 'golgi', 'microglia', 'chikv']


Top 10 words for topic #7:
['pinene', 'sporotrichosis', 'cdc42', 'fty720', 'pev', 'sab', 'g1670a', 'z2', 'a129', 'diii']


T

In [47]:
topic_values_tfidf = LDA_Tfidf.transform(doc_term_matrix_Tfidf)
topic_values_tfidf.shape

(10000, 50)

In [55]:
print(type(topic_values_tfidf))
df_topic_values_tfidf = pd.DataFrame(topic_values_tfidf)

<class 'numpy.ndarray'>


In [56]:
df_topic_values_tfidf.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.001562,0.001562,0.001562,0.001562,0.001562,0.378510,0.009503,0.001562,0.001562,0.001562,...,0.001562,0.001562,0.001562,0.001562,0.001562,0.001562,0.001562,0.001562,0.001562,0.001562
1,0.002012,0.002012,0.002012,0.044324,0.002012,0.010554,0.002012,0.002012,0.002012,0.002012,...,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012
2,0.001524,0.001524,0.022740,0.001524,0.001524,0.160525,0.035983,0.001524,0.001524,0.001524,...,0.001524,0.001524,0.001524,0.001524,0.001524,0.001524,0.001524,0.001524,0.001524,0.001524
3,0.001314,0.001314,0.001314,0.001314,0.001314,0.348331,0.018190,0.001314,0.001314,0.001314,...,0.001314,0.001314,0.001314,0.018697,0.005784,0.001314,0.001314,0.001314,0.001314,0.001314
4,0.001592,0.001592,0.001592,0.001592,0.001592,0.038035,0.007470,0.013788,0.001592,0.001592,...,0.001592,0.001592,0.001592,0.001592,0.001592,0.001592,0.001592,0.001592,0.001592,0.001592


In [45]:
for i, (topic_count_vect, topic_tfidf) in enumerate(zip(LDA.components_, LDA_Tfidf.components_)):
    print(f'Top 10 words for topic #{i} from CountVectoizer:')
    print([count_vect.get_feature_names()[i] for i in topic_count_vect.argsort()[-10:]])
    print(f'Top 10 words for topic #{i} from Tfidf:')
    print([tfidf_vect.get_feature_names()[i] for i in topic_tfidf.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0 from CountVectoizer:
['viral', 'structure', 'pcv2', 'replication', 'vesicle', 'infected', 'virus', 'cell', 'membrane', 'prrsv']
Top 10 words for topic #0 from Tfidf:
['per1', 'gnp', 'hace1', 'chikf', 'tocs', 'pakit03', 'letx', 'reinitiation', 'gemcitabine', 'psgs']


Top 10 words for topic #1 from CountVectoizer:
['stem', 'ribosome', 'frame', 'trna', 'frameshifting', 'sequence', 'translation', 'site', 'mrna', 'codon']
Top 10 words for topic #1 from Tfidf:
['dcpep', 'coe', 'odns', 'gadd45b', 'l393', 'drill', 'ppg', 'egcg', 'egr', 'emodin']


Top 10 words for topic #2 from CountVectoizer:
['cell', 'genome', 'interaction', 'ifitm3', 'host', 'replication', 'domain', 'viral', 'virus', 'protein']
Top 10 words for topic #2 from Tfidf:
['whcv', 'sod1', 'cpps', 'dnsp16', 'restv', 'igf', 'iecs', 'cccdna', 'sudv', 'spherule']


Top 10 words for topic #3 from CountVectoizer:
['method', 'positive', 'probe', 'reaction', 'pcr', 'rt', 'detection', 'primer', 'assay', 'sample']

['dvgs', 'indirubin', 'charadriiformes', 'quercetin', 'rhifitm3', 'rpfs', 'limd1', 'g64s', 'isg20', 'fcrn']


Top 10 words for topic #29 from CountVectoizer:
['solution', 'detection', 'phage', 'image', 'sample', 'method', 'temperature', 'concentration', 'surface', 'particle']
Top 10 words for topic #29 from Tfidf:
['mshmp', 'artis', 'phytol', 'sgiv', 'hais', 'richards', 'facemask', 'ndm', 'alri', 'hbov1']


Top 10 words for topic #30 from CountVectoizer:
['virus', 'domain', 'transfected', 'viral', 'expression', 'plasmid', 'binding', 'mutant', 'cell', 'protein']
Top 10 words for topic #30 from Tfidf:
['cakovs', 'ridership', 'foxo3a', '4glcnacb1', 'pmmv', 'vpu', 'rebov', 'ski', 'pcv3a', 'pcv3']


Top 10 words for topic #31 from CountVectoizer:
['genetic', 'population', 'sequence', 'position', 'allele', 'rabv', 'strain', 'rabies', 'rvfv', 'variant']
Top 10 words for topic #31 from Tfidf:
['qlamp', 'tiger', '1h', 'aoaa', 'gldc', 'leopard', 'rzikv', 'ndiv', 'vidisca', 'ly6e']


Top 10 words

In [122]:
df_covid_for_nlp_10K['topic_count_vector'] = topic_values_countvectorizer.argmax(axis=1)
df_covid_for_nlp_10K['topic_tfidf'] = topic_values_tfidf.argmax(axis=1)

In [123]:
df_covid_for_nlp_10K.head(3)

,paper_id,abstract,body_text,abstract_word_count,body_word_count,cleaned_text,topic_count_vector,topic_tfidf
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 text word count: 5168 23 24...,"vp3, and vp0 (which is further processed to vp...",241,1728,processed virus assembly region encode non-str...,30,22
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,,the 2019-ncov epidemic has spread across china...,0,755,2019-ncov epidemic spread across china country...,38,25
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,infectious bronchitis (ib) causes significant ...,"infectious bronchitis (ib), which is caused by...",1647,4003,infectious bronchitis caused infectious bronch...,3,22


In [113]:
dict_topic = {'topic_number_count_vector': [], 'topic_words_count_vector': []}

for i,topic in enumerate(LDA.components_):
    dict_topic['topic_number_count_vector'].append(i)
    dict_topic['topic_words_count_vector'].append([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])

df_covid_topics_count_vector = pd.DataFrame(dict_topic, columns=['topic_number_count_vector', 'topic_words_count_vector'])
df_covid_topics_count_vector.head()

,topic_number_count_vector,topic_words_count_vector
0,0,"[viral, structure, pcv2, replication, vesicle,..."
1,1,"[stem, ribosome, frame, trna, frameshifting, s..."
2,2,"[cell, genome, interaction, ifitm3, host, repl..."
3,3,"[method, positive, probe, reaction, pcr, rt, d..."
4,4,"[host, immune, activation, ifn, type, protein,..."


In [114]:
dict_topic = {'topic_number_tfidf': [], 'topic_words_tfidf': []}

for i,topic in enumerate(LDA_Tfidf.components_):
    dict_topic['topic_number_tfidf'].append(i)
    dict_topic['topic_words_tfidf'].append([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])

df_covid_topics_tfidf = pd.DataFrame(dict_topic, columns=['topic_number_tfidf', 'topic_words_tfidf'])
df_covid_topics.head()

,topic_number,topic_words
0,0,"[per1, gnp, hace1, chikf, tocs, pakit03, letx,..."
1,1,"[dcpep, coe, odns, gadd45b, l393, drill, ppg, ..."
2,2,"[whcv, sod1, cpps, dnsp16, restv, igf, iecs, c..."
3,3,"[ifi204, outsourcing, mdi, anguilla, rhmpv, rb..."
4,4,"[vhhs, avan, samhd1, ridd, hsp27, ccovs, eplex..."


In [124]:
df_covid_for_nlp_10K = df_covid_for_nlp_10K.merge(df_covid_topics_count_vector, how='left', left_on='topic_count_vector', right_on='topic_number_count_vector')

In [125]:
df_covid_for_nlp_10K = df_covid_for_nlp_10K.merge(df_covid_topics_tfidf, how='left', left_on='topic_tfidf', right_on='topic_number_tfidf')

In [126]:
df_covid_for_nlp_10K.head()

,paper_id,abstract,body_text,abstract_word_count,body_word_count,cleaned_text,topic_count_vector,topic_tfidf,topic_number_count_vector,topic_words_count_vector,topic_number_tfidf,topic_words_tfidf
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 text word count: 5168 23 24...,"vp3, and vp0 (which is further processed to vp...",241,1728,processed virus assembly region encode non-str...,30,22,30,"[virus, domain, transfected, viral, expression...",22,"[read, primer, sample, strain, pedv, virus, ge..."
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,,the 2019-ncov epidemic has spread across china...,0,755,2019-ncov epidemic spread across china country...,38,25,38,"[covid, period, 19, china, day, medrxiv, trans...",25,"[epidemic, disease, medrxiv, model, hospital, ..."
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,infectious bronchitis (ib) causes significant ...,"infectious bronchitis (ib), which is caused by...",1647,4003,infectious bronchitis caused infectious bronch...,3,22,3,"[method, positive, probe, reaction, pcr, rt, d...",22,"[read, primer, sample, strain, pedv, virus, ge..."
3,0139ea4ca580af99b602c6435368e7fdbefacb03,nipah virus (niv) came into limelight recently...,nipah is an infectious negative-sense single-s...,326,2399,nipah infectious negative-sense single-strande...,45,5,45,"[activity, complex, interaction, acid, protein...",5,"[il, cov, response, expression, antibody, vira..."
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,background: a novel coronavirus (2019-ncov) em...,"in december 2019, a cluster of patients with p...",22,4642,december 2019 cluster patient pneumonia unknow...,38,25,38,"[covid, period, 19, china, day, medrxiv, trans...",25,"[epidemic, disease, medrxiv, model, hospital, ..."


In [128]:
df_covid_for_nlp_10K.columns

Index(['paper_id', 'abstract', 'body_text', 'abstract_word_count',
       'body_word_count', 'cleaned_text', 'topic_count_vector', 'topic_tfidf',
       'topic_number_count_vector', 'topic_words_count_vector',
       'topic_number_tfidf', 'topic_words_tfidf'],
      dtype='object')

In [136]:
# df_covid_for_nlp_10K[['paper_id', 'body_text', 'body_word_count', 'topic_count_vector', 'topic_words_count_vector', 'topic_tfidf', 'topic_words_tfidf']].head()
# df_covid_for_nlp_10K[['paper_id', 'body_text', 'body_word_count', 'topic_count_vector', 'topic_words_count_vector', 'topic_tfidf', 'topic_words_tfidf']].to_csv(output / 'df_covid_for_nlp_10K.csv', index = False)
# df_covid_for_nlp_10K[['paper_id', 'topic_count_vector', 'topic_words_count_vector', 'topic_tfidf', 'topic_words_tfidf', 'cleaned_text', 'body_word_count']].to_csv(output / 'df_covid_for_nlp_10K.csv', index = False)

df_covid_for_nlp_10K[['paper_id', 'topic_count_vector', 'topic_words_count_vector', 'topic_tfidf', 'topic_words_tfidf', 'body_word_count']].to_csv(output / 'df_covid_for_nlp_10K.csv', index = False)

In [137]:
df_all_sources_metadata['sha'].value_counts()

4644c32551fb23aa873a7738ecc8d777bd49877e    4
72a5640aa0c307fbe171ca7ad55d3fda48b53988    4
9ce0a6cfd53840cd985f7a1439708c7a48bb7f23    3
f3aafdecdc43a3f57e58cf6dcea038b1834a953e    2
58be092086c74c58e9067121a6ba4836468e7ec3    2
                                           ..
129341a52b43ccd59368d094950322d983ebe223    1
88def2a04604c4a9a918c9273bfc8d336323045f    1
92b28fb0a43d34081844407727df1dbd2474c7ab    1
b59b58c13370a390a58c6cd79e61f97bf9bee37d    1
4bf587d8051a33e9e8215ef96afa025a943dc1b8    1
Name: sha, Length: 28450, dtype: int64

In [159]:
df_all_sources_metadata['full_text_file'].value_counts()

custom_license        20873
comm_use_subset        8803
noncomm_use_subset     2133
biorxiv_medrxiv        1020
Name: full_text_file, dtype: int64

In [164]:
# pd.pivot_table(df_all_sources_metadata, index='full_text_file', values='sha', aggfunc='count', margins=True)

pd.pivot_table(df_all_sources_metadata, index='full_text_file', values=['sha', 'source_x'], aggfunc='count', margins=True)

,sha,source_x
full_text_file,,
biorxiv_medrxiv,885,1020
comm_use_subset,8695,8803
custom_license,16861,20873
noncomm_use_subset,2021,2133
All,28462,28462


In [150]:
df_all_sources_metadata_with_topics = df_all_sources_metadata.copy()
df_all_sources_metadata_with_topics.shape

(44220, 15)

In [148]:
# df_all_sources_metadata_with_topics = df_all_sources_metadata_with_topics.merge(df_covid_for_nlp_10K[['paper_id', 'topic_count_vector', 'topic_words_count_vector', 'topic_tfidf', 'topic_words_tfidf', 'body_word_count']], how='left', left_on='sha', right_on='paper_id')

df_all_sources_metadata_with_topics = df_all_sources_metadata_with_topics.merge(df_covid_for_nlp_10K, how='left', left_on='sha', right_on='paper_id')

In [149]:
print(df_all_sources_metadata_with_topics.columns)
print(df_all_sources_metadata_with_topics.shape)
df_all_sources_metadata_with_topics.head()

Index(['sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id', 'license',
       'abstract_x', 'publish_time', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'has_full_text',
       'full_text_file', 'paper_id', 'abstract_y', 'body_text',
       'abstract_word_count', 'body_word_count', 'cleaned_text',
       'topic_count_vector', 'topic_tfidf', 'topic_number_count_vector',
       'topic_words_count_vector', 'topic_number_tfidf', 'topic_words_tfidf'],
      dtype='object')
(44220, 27)


,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract_x,publish_time,authors,...,body_text,abstract_word_count,body_word_count,cleaned_text,topic_count_vector,topic_tfidf,topic_number_count_vector,topic_words_count_vector,topic_number_tfidf,topic_words_tfidf
0,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535.0,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850.0,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701.0,els-covid,NaN,1980-03-31,"Friedman, Gary D",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077.0,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285.0,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
df_all_sources_metadata_with_topics['topic_count_vector'].value_counts(dropna=False)

NaN     34976
35.0      590
10.0      458
6.0       416
27.0      402
25.0      382
38.0      342
46.0      340
41.0      333
30.0      328
9.0       311
4.0       295
3.0       288
8.0       243
33.0      232
24.0      229
23.0      211
45.0      211
47.0      205
44.0      205
48.0      200
5.0       195
32.0      177
29.0      177
28.0      176
7.0       175
12.0      174
36.0      164
39.0      160
22.0      157
2.0       150
42.0      149
40.0      119
15.0      118
16.0       96
17.0       80
11.0       80
20.0       71
37.0       70
14.0       68
21.0       66
19.0       61
1.0        54
26.0       48
0.0        45
49.0       43
18.0       39
34.0       39
43.0       28
13.0       22
31.0       22
Name: topic_count_vector, dtype: int64